In [ ]:
import numpy as np
from scipy.stats import norm

from run import run


In [ ]:
'''generate data without error'''

def get_data(n, seed_0, k1, k2, intercept, slope):
    # define mean and covariance matrix for mutiariate normal vector
    # n: sample size
    # k1: dim of continuous covariate vector
    # k2: dim of discrete covariate vector
    # intercept: scalar
    # slope: (p,)

    mean = np.zeros(k1)
    cov = np.ones((k1, k1))
    
    for i in range(k1):
        for j in range(k1):
            cov[i, j] = 0.5 ** (abs(i - j))
    
    np.random.seed(seed_0)
    Z1 = np.random.multivariate_normal(mean, cov, n)

    np.random.seed(seed_0 * 2)
    Z2 = np.random.binomial(1, 0.5, (n, k2))


    Z = np.concatenate((Z1, Z2), axis=1)
    
    logi = (intercept + np.matmul(Z, slope)).reshape(-1) # (n, p) * (p, 1) -> (n, 1) -> (n,)
    p = np.exp(logi) / (1 + np.exp(logi))
    Y = np.zeros(n)
    for i in range(n):
        np.random.seed(i + 10)
        if np.random.rand() < p[i]:
            Y[i] = 1
    
    return Z, Y

'''generate noisy label'''

def noisy_label(Z, Y, r, b0, b, loc0):
    '''
        input:
            Z -- covariates
            Y -- true label vector
            r -- model misspecification parameter
        output:
            Y_star -- noisy label
    '''
    
    n = Y.shape[0]
    Y_star = np.copy(Y)

    for i in range(n):
        p1 = norm.cdf(Z[i, 1] ** 2, loc=loc0)
        p2 = np.exp(b0 + np.dot(Z[i, :], b)) / (1 + np.exp(b0 + np.dot(Z[i, :], b)))
        p = r * p1 + (1 - r) * p2
        
        np.random.seed(n + i + 100)
        if np.random.rand() < p:
            Y_star[i] = 1 - Y[i]
                
    return Y_star

In [ ]:
n = 1000  # sample size
p = 20 # number of covariates
delta = 0.3 # validation ratio

intercept = 1
slope = np.zeros((p,))
slope[0], slope[1], slope[4], slope[5], slope[9] = 2, 1.3, 2, -1.5, 1 

b =np.zeros((p,))
b[0], b[1], b[2], b[3], b[4] = 1, 1, -1.5, 1.1, -1.3

seed = 1
Z_, Y_ = get_data(n, seed_0 = seed, k1=18, k2=2, intercept=intercept, slope=slope) 
Y_star_ = noisy_label(Z_, Y_, r=0, b0=-2.15, b=b, loc0=1.98) 


n_v = int(np.floor(n * delta))

Z_val = Z_[:n_v, :]
Y_star_val = Y_star_[:n_v]
Y_val = Y_[:n_v]

Z = Z_[n_v:, :]
Y_star = Y_star_[n_v:]

seed = 2
Z_test, Y_test = get_data(n, seed_0 = seed, k1=18, k2=2, intercept=intercept, slope=slope) 


In [ ]:
result = run(Z, Y_star, Z_val, Y_val, Y_star_val, discrete_idx=[18, 19], Z_test=Z_test, Y_test=Y_test, test=True,
        link_func='logit', penalty='scad', use_intercept=True, criterion='gcv',
        model_running='semi', densityType='Kernel', 
        eta=0.91, R=5, L=0.05, N_iter=3, max_loop=10
        )
print(result)